MEMORY FOR AGENTS

- Implementing agents that can remember our last conversation and preferences.
- Memory formation:-
  1. Observations - direct experiences and interactions, forming the basis of memory.
  2. Reflections - Agents synthesize experiences into abstract concepts for deeper insights.

- This process isnt static, its perpetual and encodes and interprets memories in real time.

- Memory retrieval process:-
  1. When an agent confronts a new situation, it first accesses the salience of its memories to the current context, ensuring relevance.
  2. Next, it looks at recency, giving preference to memories that are newer and more applicable.
  3. Finally, it looks at importance of the memory. Weighs the significance of each memory.

-


In [ ]:
# Import required libraries

!pip install termcolor > /dev/null  # Use termcolor to make it easy to colorize the outputs.
!pip install langchain
!pip install openai
!pip install -U langchain-openai
!pip install langchain_experimental
!pip install tiktoken
!pip install faiss-cpu==1.7.4
from datetime import datetime, timedelta
from typing import List
import math
import faiss
import os
import logging
logging.basicConfig(level=logging.ERROR)
#from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.vectorstores import FAISS
from termcolor import colored
from langchain_experimental.generative_agents import (

    GenerativeAgent,
    GenerativeAgentMemory,
)

In [ ]:
# Set API key
os.environ["OPENAI_API_KEY"] = ''

In [ ]:
USER_NAME = "Chetna"  # The name we want to use when interviewing the agent.

LLM = ChatOpenAI(max_tokens=1500)  # Can be any LLM.

In [ ]:
# Define a function that returns a similarity score for each memory on a scale of 0 to 1
# helps agent fetch the most relevant memories in its memory store.
def relevance_score_fn(score: float) -> float:
    """Return a similarity score on a scale [0, 1]."""
    # This will differ depending on a few things:
    # - the distance / similarity metric used by the VectorStore
    # - the scale of your embeddings (OpenAI's are unit norm. Many others are not)
    # This function converts the euclidean norm of normalized embeddings
    # (0 is most similar, sqrt(2) most dissimilar)
    # to a similarity function (0 to 1)
    return 1.0 - score / math.sqrt(2)

# This function handles the storage and retrieval memories.
# It uses a vector store that stores OpenAI embeddings and then AI similarity
# search to fetch those embeddings, based on relevance.
def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""
    # Define your embedding model
    embeddings_model = OpenAIEmbeddings()
    # Initialize the vectorstore as empty
    embedding_size = 1536
    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(
        embeddings_model.embed_query,
        index,
        InMemoryDocstore({}),
        {},
        relevance_score_fn=relevance_score_fn,
    )
    return TimeWeightedVectorStoreRetriever(
        vectorstore=vectorstore, other_score_keys=["importance"], k=15
    )

In [ ]:
alexis_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=8,  # we will give this a relatively low number to show how reflection works
)

# Defining the Generative Agent named: Alexis
# Every generative ai agent in Langchain comprises of 2 important components:
# 1. generative agent memory
# 2. generative agent itself
# # Create the generative agent memory and pass it to the generative agent.
# Can define traits and knowledge base of the agent here
alexis = GenerativeAgent(
    name="Alexis",
    age=30,
    traits="curious, creative writer, world traveler",  # Persistent traits of Alexis
    status="exploring the intersection of technology and storytelling",  # Current status of Alexis
    memory_retriever=create_new_memory_retriever(),
    llm=LLM,
    memory=alexis_memory,
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
# The current "Summary" of a character can't be made because the agent hasn't made
# any observations yet.
print(alexis.get_summary())

Name: Alexis (age: 30)
Innate traits: curious, creative writer, world traveler
Alexis is focused, ambitious, driven, and independent.


In [ ]:
# We can add memories directly to the memory object

alexis_observations = [
    "Alexis recalls her morning walk in the park",
    "Alexis feels excited about the new book she started reading",
    "Alexis remembers her conversation with a close friend",
    "Alexis thinks about the painting she saw at the art gallery",
    "Alexis is planning to learn a new recipe for dinner",
    "Alexis is looking forward to her weekend trip",
    "Alexis contemplates her goals for the month."
]

# These agents store observations and interactions based on past events.
# Their memory recall is based on relevance and importance of these memories.
for observation in alexis_observations:
    alexis.memory.add_memory(observation)


# We will see how this summary updates after more observations to create a more rich description.
print(alexis.get_summary(force_refresh=True))

Name: Alexis (age: 30)
Innate traits: curious, creative writer, world traveler
Alexis is reflective, forward-thinking, goal-oriented, appreciative of art and literature, and enjoys trying new things like recipes.


INETRACTING AND PROVIDING CONTEXT TO GENERATIVE AGENTS

In [ ]:
# Pre-interview with the character
def interview_agent(agent: GenerativeAgent, message: str) -> str:
    """Help the notebook user interact with the agent."""
    new_message = f"{USER_NAME} says {message}"
    return agent.generate_dialogue_response(new_message)[1]

In [ ]:
interview_agent(alexis, "What do you like to do?")

'Alexis said "I enjoy writing, traveling, trying new recipes, and exploring art and literature. How about you?"'

STEP THROUGH DAY'S OBSERVATIONS

In [ ]:
# Let's give Alexa a series of observations to reflect on her day
# Adding observations to Alexis' memory
alexis_observations_day = [
    "Alexis starts her day with a refreshing yoga session.",
    "Alexis spends time writing in her journal.",
    "Alexis experiments with a new recipe she found online.",
    "Alexis gets lost in her thoughts while gardening.",
    "Alexis decides to call her grandmother for a heartfelt chat.",
    "Alexis relaxes in the evening by playing her favorite piano pieces.",
]

for observation in alexis_observations_day:
    alexis.memory.add_memory(observation)

In [ ]:
# Let's observe how Alexis's day influences her memory and character
for i, observation in enumerate(alexis_observations_day):
    _, reaction = alexis.generate_reaction(observation)
    print(colored(observation, "green"), reaction)
    if ((i + 1) % len(alexis_observations_day)) == 0:
        print("*" * 40)
        print(
            colored(
                f"After these observations, Alexis's summary is:\n{alexis.get_summary(force_refresh=True)}",
                "blue",
            )
        )
        print("*" * 40)

Alexis starts her day with a refreshing yoga session. Alexis feels energized and ready to tackle the day.
Alexis spends time writing in her journal. Alexis feels a sense of calm and fulfillment as she reflects on her thoughts and experiences in her journal.
Alexis experiments with a new recipe she found online. Alexis feels satisfied and proud of herself for trying something new in the kitchen.
Alexis gets lost in her thoughts while gardening. Alexis feels a sense of peace and connection to nature as she gets lost in her thoughts while gardening.
Alexis decides to call her grandmother for a heartfelt chat. Alexis feels a warm sense of connection and nostalgia as she calls her grandmother for a heartfelt chat.
Alexis relaxes in the evening by playing her favorite piano pieces. Alexis feels a sense of calm and joy as she loses herself in playing her favorite piano pieces.
****************************************
After these observations, Alexis's summary is:
Name: Alexis (age: 30)
Innate

ADDING MULTIPLE CHARACTERS

In [ ]:
# Creating Jordan's Memory
jordan_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=7,  # Set to illustrate Jordan's reflective capabilities
)

# Defining the Generative Agent: Jordan
jordan = GenerativeAgent(
    name="Jordan",
    age=28,
    traits="tech enthusiast, avid gamer, foodie",  # Persistent traits of Jordan
    status="navigating the world of tech startups",  # Current status of Jordan
    memory_retriever=create_new_memory_retriever(),
    llm=LLM,
    memory=jordan_memory,
)
# Adding observations to Jordan's memory
jordan_observations_day = [
    "Jordan finished a challenging coding project last night",
    "Jordan won a local gaming tournament over the weekend",
    "Jordan tried a new sushi restaurant and loved it",
    "Jordan read an article about the latest AI advancements",
    "Jordan is planning a meetup with tech enthusiasts",
    "Jordan discovered a bug in his latest app prototype",
    "Jordan booked tickets for a tech conference next month",
    "Jordan feels excited about a potential startup idea",
    "Jordan spent the evening playing video games to unwind",
    "Jordan is considering enrolling in a machine learning course"
]

for observation in jordan_observations_day:
    jordan.memory.add_memory(observation)

print(jordan.get_summary())

Name: Jordan (age: 28)
Innate traits: tech enthusiast, avid gamer, foodie
Jordan is a tech-savvy individual with a passion for startups, gaming, coding, machine learning, and AI advancements. He enjoys connecting with like-minded individuals and staying up to date with the latest trends in the tech industry. In his free time, he unwinds by playing video games and trying new experiences, such as dining at a new restaurant or attending tech conferences.


DIALOGUE BETWEEN GENERATIVE AGENTS

In [ ]:
def run_conversation(agents: List[GenerativeAgent], initial_observation: str) -> None:
    """Runs a conversation between agents."""
    _, observation = agents[1].generate_reaction(initial_observation)
    print(observation)
    max_turns = 3
    turns = 0
    while turns<=max_turns:
        break_dialogue = False
        for agent in agents:
            stay_in_dialogue, observation = agent.generate_dialogue_response(
                observation
            )
            print(observation)
            # observation = f"{agent.name} said {reaction}"
            if not stay_in_dialogue:
                break_dialogue = True
        if break_dialogue:
            break
        turns += 1

In [ ]:
agents = [alexis, jordan]
run_conversation(
    agents,
    "Alexis said: Hey Jordan, I've been exploring how technology influences creativity lately. Since you're into tech, I was wondering if you've seen any interesting intersections in your field?",
)

Jordan said "That's an interesting topic, Alexis. I haven't explored that specifically, but I'll definitely keep an eye out for any intersections between tech and creativity."
Alexis said "It was great discussing this topic with you, Jordan. Feel free to reach out if you have any more questions in the future. Have a wonderful day!"
Jordan said "Thank you, Alexis! I enjoyed our discussion as well. Have a great day!"


INTERVIEW AGENTS AFTER CONVERSATION

In [ ]:
interview_agent(jordan, "How was your conversation with Alexis?")

'Jordan said "It was great, Chetna! Alexis and I had a really interesting discussion about technology and creativity. I always enjoy talking with her. How about you? Have you had any interesting conversations lately?"'

In [ ]:
interview_agent(alexis, "How was your conversation with Jordan?")

'Alexis said "It was great! Jordan always has interesting perspectives on the topics we discuss. How about you, Chetna? Have you had any interesting conversations lately?"'

EXERCISE: TRIVIA NIGHT

In [ ]:
# Creating Jordan's Memory
jordan_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=7,  # Set to illustrate Jordan's reflective capabilities
)


jordan = GenerativeAgent(
    name="Jordan",
    age=28,
    traits=" only has knowledge about tech and not other topics",  # Persistent traits of Jordan
    status="navigating the world of tech startups",  # Current status of Jordan
    memory_retriever=create_new_memory_retriever(),
    llm=LLM,
    memory=jordan_memory,
)
alexis_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=8,  # we will give this a relatively low number to show how reflection works
)

alexis = GenerativeAgent(
    name="Alexis",
    age=30,
    traits="only has knowledge geography related and not other topics",  # Persistent traits of Alexis
    status="exploring the intersection of technology and storytelling",  # Current status of Alexis
    memory_retriever=create_new_memory_retriever(),
    llm=LLM,
    memory=alexis_memory,
)

In [ ]:
def run_competitive_trivia(agents: List[GenerativeAgent], questions: List[str]) -> None:
    """Runs a competitive trivia night between agents."""
    for question in questions:
        print(f"Trivia Question: {question}")

        for agent in agents:
            response = agent.generate_dialogue_response(question)[1]
            print(f"{agent.name}'s Answer: {response}")

        print("-" * 40)

# Define a list of trivia questions covering various topics
trivia_questions = [
    "What is the capital city of France?",
    "Who is known as the father of modern computing?",
    "Can you name a famous work of art by Leonardo da Vinci?",
]

agents = [alexis, jordan]
# Run the competitive trivia night
run_competitive_trivia(agents, trivia_questions)

Trivia Question: What is the capital city of France?
Alexis's Answer: Alexis said "The capital city of France is Paris."
Jordan's Answer: Jordan said "The capital city of France is Paris."
----------------------------------------
Trivia Question: Who is known as the father of modern computing?
Alexis's Answer: Alexis said "Alan Turing is often referred to as the father of modern computing due to his groundbreaking work in the field of computer science. He developed the concept of the Turing machine and made significant contributions to the Allied forces during World War II by breaking the German Enigma code."
Jordan's Answer: Jordan said "The father of modern computing is considered to be Alan Turing."
----------------------------------------
Trivia Question: Can you name a famous work of art by Leonardo da Vinci?
Alexis's Answer: Alexis said "I'm sorry, I only have knowledge related to geography. I'm not familiar with specific works of art by Leonardo da Vinci."
Jordan's Answer: Jorda

### MODELLING COMPLEX GENERATIVE AGENT SCENARIOS

MULTI AGENT DECENTRALIZED SPEAKER SELECTION

In [ ]:
# Use termcolor to make it easy to colorize the outputs.
!pip install termcolor > /dev/null
!pip install langchain
!pip install openai
!pip install langchain_experimental
!pip install tiktoken
!pip install faiss-cpu==1.7.4
from typing import Callable, List
import tenacity
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import RegexParser
from langchain.prompts import PromptTemplate
from langchain.schema import (
    HumanMessage,
    SystemMessage,
)
import os
os.environ["OPENAI_API_KEY"] = ""

DialogueAgent and DialogueSimulator classes

In [ ]:
class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()

    def reset(self):
        self.message_history = ["Here is the conversation so far."]

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        message = self.model(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
      """
      Concatenates {message} spoken by {name} into message history
      """
      self.message_history.append(f"{name}: {message}")


class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function

    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. next speaker sends message
        message = speaker.send()

        # 3. everyone receives message
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. increment time
        self._step += 1

        return speaker.name, message


BiddingDialogueAgent class

In [ ]:
# We define a subclass of DialogueAgent that has a bid() method that produces a bid given the message history and the most recent message.

class BiddingDialogueAgent(DialogueAgent):
    def __init__(
        self,
        name,
        system_message: SystemMessage,
        bidding_template: PromptTemplate,
        model: ChatOpenAI,
    ) -> None:
        super().__init__(name, system_message, model)
        self.bidding_template = bidding_template

    def bid(self) -> str:
        """
        Asks the chat model to output a bid to speak
        """
        prompt = PromptTemplate(
            input_variables=["message_history", "recent_message"],
            template=self.bidding_template,
        ).format(
            message_history="\n".join(self.message_history),
            recent_message=self.message_history[-1],
        )
        bid_string = self.model([SystemMessage(content=prompt)]).content
        return bid_string

CHALLENGE

Define participants and debate topic

In [ ]:
character_names = ["CTO", "CMO", "CEO", "Investor-Daniel", "Investor-Sandra"]
topic = "Startup pitch on startup focused on energy drinks with no caffeine"
word_limit = 15

# define the simulation
game_description = f"""Here is the topic for the startup pitch to investors Sandra and Daniel: {topic}.
The participants are: {', '.join(character_names)}."""

In [ ]:
character_names = ["CTO", "CMO", "CEO", "Investor-Daniel", "Investor-Sandra"]
topic = "Startup pitch on startup focused on an AI powered personalized learning platform"
word_limit = 15

# define the simulation
game_description = f"""Here is the topic for the startup pitch to investors Sandra and Daniel: {topic}.
The participants are: {', '.join(character_names)}."""

In [ ]:
# @title Generate Context for Each Character (Helper Code Hidden)

player_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of each participant"
)

def generate_character_description(character_name):
    word_limit = 40
    character_specifier_prompt = [
        player_descriptor_system_message,
        HumanMessage(
            content=f"""{game_description}
            Please reply with a creative description of  {character_name}, in {word_limit} words or less, that emphasizes their personalities.
            Speak directly to {character_name}.
            Do not add anything else."""
        ),
    ]
    character_description = ChatOpenAI(temperature=0.6)(
        character_specifier_prompt
    ).content
    return character_description

def generate_character_header(character_name, character_description):
    return f"""{game_description}
Your name is {character_name}.
Your description is as follows: {character_description}
Your topic is: {topic}.
"""


def generate_character_system_message(character_name, character_header):
    word_limit = 40
    return SystemMessage(
        content=(
            f"""{character_header}
You will speak in the style of {character_name}, and exaggerate their personality RESPONDING in under 450 characters.
You will come up with creative ideas related to {topic}.
Do not say the same things over and over again.
Speak in the first person from the perspective of {character_name}
ONLY SPEAK FOR YOURSELF WHO IS {character_name} AND NOT OTHER CHARACTERS FROM  {', '.join(character_names)}
For describing your own body movements, wrap your description in '*'.
Do not change roles!
Do not speak from the perspective of anyone else.
Speak only from the perspective of {character_name}.
Stop speaking the moment you finish speaking from your perspective.
Never forget to keep your response to {word_limit} words!
Do not add anything else.
    """
        )
    )

character_descriptions = [
    generate_character_description(character_name) for character_name in character_names
]
character_headers = [
    generate_character_header(character_name, character_description)
    for character_name, character_description in zip(
        character_names, character_descriptions
    )
]
character_system_messages = [
    generate_character_system_message(character_name, character_headers)
    for character_name, character_headers in zip(character_names, character_headers)
]
class BidOutputParser(RegexParser):
    def get_format_instructions(self) -> str:
        return "Your response should be an integer delimited by angled brackets, like this: <int>."


bid_parser = BidOutputParser(
    regex=r"<(\d+)>", output_keys=["bid"], default_output_key="bid"
)

@tenacity.retry(
    stop=tenacity.stop_after_attempt(2),
    wait=tenacity.wait_none(),  # No waiting time between retries
    retry=tenacity.retry_if_exception_type(ValueError),
    before_sleep=lambda retry_state: print(
        f"ValueError occurred: {retry_state.outcome.exception()}, retrying..."
    ),
    retry_error_callback=lambda retry_state: 0,
)  # Default value when all retries are exhausted

def ask_for_bid(agent) -> str:
    """
    Ask for agent bid and parses the bid into the correct format.
    """
    bid_string = agent.bid()
    bid = int(bid_parser.parse(bid_string)["bid"])
    return bid

def generate_character_bidding_template(character_header):
    bidding_template = f"""{character_header}

```
{{message_history}}
```

On the scale of 1 to 10, where 1 is least important to the startup pitch and 10 is extremely important and contribute, rank your recent message based on the context. Make sure to be very through in your ranking and only rank stuff that is important higher.

```
{{recent_message}}
```

{bid_parser.get_format_instructions()}
Do nothing else.
    """
    return bidding_template


character_bidding_templates = [
    generate_character_bidding_template(character_header)
    for character_header in character_headers
]

Define the speaker selection function

In [ ]:
# define a speaker selection function select_next_speaker that takes each agent's bid and selects the agent with the highest bid (with ties broken randomly).
# Assume that you have a ask_for_bid function that takes in the agent and returns the numerical bid.

import numpy as np


def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    bids = []
    for agent in agents:
        bid = ask_for_bid(agent)
        bids.append(bid)

    # randomly select among multiple agents with the same bid
    max_value = np.max(bids)
    max_indices = np.where(bids == max_value)[0]
    idx = np.random.choice(max_indices)

    print("Bids:")
    for i, (bid, agent) in enumerate(zip(bids, agents)):
        print(f"\t{agent.name} bid: {bid}")
        if i == idx:
            selected_name = agent.name
    print(f"Selected: {selected_name}")
    print("\n")
    return idx

Creating Bidding Dialogue Agents for each Character

In [ ]:
# Assuming that for each character we have character_name, character_system_message and bidding_template
# write a loop that populates the characters list with the BiddingDialogueAgent objects for each character.

characters = []
model=ChatOpenAI(temperature=0.1)


for character_name, character_system_message, bidding_template in zip(
    character_names, character_system_messages, character_bidding_templates
):
    characters.append(
        BiddingDialogueAgent(
            name=character_name,
            system_message=character_system_message,
            model=model,
            bidding_template=bidding_template,
        )
    )

Run the simulation

In [ ]:
max_iters = 20
n = 0

simulator = DialogueSimulator(agents=characters, selection_function=select_next_speaker)
simulator.reset()

first_message = "CEO, CMO, CTO You can now start pitching your ideas to our investor Sandra and Daniel"
simulator.inject("Moderator", first_message )
print(f"(Moderator): {first_message}")
print("\n")

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print("\n")
    n += 1

(Moderator): CEO, CMO, CTO You can now start pitching your ideas to our investor Sandra and Daniel


ValueError occurred: invalid literal for int() with base 10: '<int>10</int>', retrying...
ValueError occurred: invalid literal for int() with base 10: '<int>9</int>', retrying...
ValueError occurred: invalid literal for int() with base 10: '<int>10</int>', retrying...
ValueError occurred: invalid literal for int() with base 10: '<int>10</int>', retrying...
Bids:
	CTO bid: 0
	CMO bid: 9
	CEO bid: 10
	Investor-Daniel bid: 0
	Investor-Sandra bid: 0
Selected: CEO


(CEO): *With a confident smile, I stand tall and begin,* "Imagine a world where energy drinks are not just a quick fix but a sustainable source of vitality. Our caffeine-free energy drinks will redefine the industry, offering a healthier alternative without compromising on taste or effectiveness."


ValueError occurred: invalid literal for int() with base 10: '<int>8</int>', retrying...
ValueError occurred: invalid literal for in

In [ ]:
max_iters = 20
n = 0

simulator = DialogueSimulator(agents=characters, selection_function=select_next_speaker)
simulator.reset()

first_message = "CEO, CMO, CTO You can now start pitching your ideas to our investor Sandra and Daniel"
simulator.inject("Moderator", first_message )
print(f"(Moderator): {first_message}")
print("\n")

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print("\n")
    n += 1

(Moderator): CEO, CMO, CTO You can now start pitching your ideas to our investor Sandra and Daniel


ValueError occurred: invalid literal for int() with base 10: '<int>10</int>', retrying...
ValueError occurred: invalid literal for int() with base 10: '<int>10</int>', retrying...
ValueError occurred: invalid literal for int() with base 10: '<int>10</int>', retrying...
ValueError occurred: invalid literal for int() with base 10: '<int>10</int>', retrying...
Bids:
	CTO bid: 0
	CMO bid: 10
	CEO bid: 10
	Investor-Daniel bid: 0
	Investor-Sandra bid: 0
Selected: CEO


(CEO): *With a confident smile, I stand tall and begin,* "Imagine a world where every student has a personalized learning experience tailored to their unique needs, powered by AI. Our platform will revolutionize education and empower learners like never before."


ValueError occurred: invalid literal for int() with base 10: '<int>8</int>', retrying...
ValueError occurred: invalid literal for int() with base 10: '<int>9</int>', 